# Confined Aquifer Model

Hydrogeology Field Camp
ESci 4971W/5971, University of Minnesota


In [ ]:
# import python packages
import pathlib as pl
import scipy.special
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import flopy

## Load Data

Well and drawdown data is included in the ./data folder.  The following jupyter notebook cells in this section load these data files into Python.

In [ ]:
f = pl.Path("./data/well_data.csv")
wells_df = pd.read_csv(f)
wells_df = wells_df.set_index("name")
# wells_df

In [ ]:
# Create a function that returns a dictionary of well attributes
def get_well_data(name):
    return wells_df.loc[name].to_dict()

# Show what the function returns
get_well_data("HB-1")

In [ ]:
# plot well locations
ax = wells_df.plot.scatter(x="easting(m)", y="northing(m)")

In [ ]:
# Assign datetime value to the start of the pumping test
format = "%Y-%m-%d %H:%M:%S"

# # 2023
# start_date_time = "2023-08-02 10:52:00"
# start_datetime = datetime.strptime(start_date_time, format)
# f = pl.Path("./data/hb2_drawdown2023.csv")
# hb2_df = pd.read_csv(f, parse_dates=["date_time"])

# 2024
start_date_time = "2024-07-31 10:56:00" # 2024
start_datetime = datetime.strptime(start_date_time, format)
f = pl.Path("./data/hb2_drawdown2024.csv")
hb2_df = pd.read_csv(f, parse_dates=["date_time"])

print(f"The pumping test started at {start_datetime=}")

# remove any entries that occured before the pump turned on
hb2_df.drop(hb2_df[hb2_df.date_time < start_datetime].index, inplace=True)

# Perform calculations in order to assign pumping time columns in minutes and days
hb2_df["pumping_time_td"] = hb2_df["date_time"] - start_datetime
hb2_df["pumping_time(min)"] = hb2_df["pumping_time_td"].dt.total_seconds() / 60
hb2_df["pumping_time(days)"] = hb2_df["pumping_time(min)"] / 1440
hb2_df["drawdown(m)"] = hb2_df["drawdown(ft)"] * 0.3048

# Plot the HB-2 measured drawdown
hb2_df.plot(x="pumping_time(min)", y="drawdown(m)")


## Create Model Grid

Let's create a MODFLOW model grid with HB-1 in the middle.  Let's use small cells in the middle of the grid and have them get larger towards the edge.  This is called a "telescoping" grid and is often used for aquifer test analysis.

In [ ]:
nlay = 1
nrow_inner = 101
ncol_inner = 101
dx = 1.0 # meters
dy = dx
dx_outer = [1.5 ** i for i in range(1, 20)]
dy_outer = dx_outer
delr = dx_outer[::-1] + ncol_inner * [dx] + dx_outer
delc = dy_outer[::-1] + nrow_inner * [dy] + dy_outer
delr = np.array(delr)
delc = np.array(delc)
nrow = delc.shape[0]
ncol = delr.shape[0]
top = 1240. * 0.3048 # top in meters
top = top * np.ones((nrow, ncol), dtype=float)
botm = 1230. * 0.3048 # bottom in meters
botm = botm * np.ones((nlay, nrow, ncol), dtype=float)

# create a flopy grid object
model_grid = flopy.discretization.StructuredGrid(
    delr=delr, 
    delc=delc, 
    top=top, 
    botm=botm
)

In [ ]:
model_grid.plot()
plt.xlabel("easting(m)")
plt.ylabel("northing(m)")

## Offset Grid

Offset the model grid into real world coordinates.

In [ ]:
w = get_well_data("HB-1")
grid_center_x, grid_center_y = w["easting(m)"], w["northing(m)"]
print(f"{grid_center_x=} {grid_center_y=}")
grid_length_x = delr.sum()
grid_length_y = delc.sum()
xoff = grid_center_x - 0.5 * grid_length_x
yoff = grid_center_y - 0.5 * grid_length_y
model_grid.set_coord_info(xoff=xoff, yoff=yoff)
ax = wells_df.plot.scatter(x="easting(m)", y="northing(m)")
model_grid.plot(ax=ax, lw=0.1)


In [ ]:
# Demonstrate how to find model row and column by name of well
for name in ["HB-1", "HB-2", "HB-3"]:
    w = get_well_data(name)
    i, j = model_grid.intersect(w["easting(m)"], w["northing(m)"])
    print(name, i, j)

In [ ]:
# # pumping information for 2023
# # pumping started on Aug 2, 2023 at 10:52 and ended Aug 4, 2023 at 14:06
# pumping_rate = 6.99 # ft^3/min
# pumping_rate *= 1 / 3.2808 ** 3 * 1440 # convert to meters per day
# print(f"{pumping_rate=:.2f} m^3/day")
# pumping_time = 53. + 14. / 60. # hours
# pumping_time *= 1 / 24.
# print(f"{pumping_time=:.2f} days")

# pumping information for 2024
# pumping started on Jul 31, 2024 at 10:56 and ended Aug 3, 2024 at 13:47
pumping_rate = 60. # gallons / min
pumping_rate *= 1. / 7.48 # ft^3/min
pumping_rate *= 1 / 3.2808 ** 3 * 1440 # convert to meters per day
print(f"{pumping_rate=:.2f} m^3/day")
pumping_time = 3.11875 # days
print(f"{pumping_time=:.2f} days")

In [ ]:
# specify hydraulic conductivity to use in model
hydraulic_conductivity = 10.0 # m / day

# specify the specific storage to use in model
specific_storage = 1.e-5

# calculate aquifer thickness, transmissivity, storage coefficient
aquifer_thickness = top[0, 0] - botm[0, 0, 0]
transmissivity = hydraulic_conductivity * aquifer_thickness
storage_coefficient = specific_storage * aquifer_thickness

print(f"{aquifer_thickness=:.2f} m")
print(f"{hydraulic_conductivity=:.2f} m/day")
print(f"{specific_storage=:.2e} 1/m")
print(f"{transmissivity=:.2f} m/day")
print(f"{storage_coefficient=:.2e} per meter")


In [ ]:
ws = './sim03-confined'
name = 'mymodel'

# create simulation object
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws, exe_name='mf6')

# set up time stepping
perlen = pumping_time
nstp = 50
tsmult = 1.2
perioddata = 1 * [(perlen, nstp, tsmult)]
nper = len(perioddata)
tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=perioddata)

# solver
ims = flopy.mf6.ModflowIms(
    sim, 
    inner_dvclose=0.0001, 
    outer_dvclose=0.0001
)

# groundwater flow model
gwf = flopy.mf6.ModflowGwf(
    sim, 
    modelname=name, 
    save_flows=True,
)

# create the discretization
dis = flopy.mf6.ModflowGwfdis(
    gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc,
    top=top, botm=botm, xorigin=xoff, yorigin=yoff
)

# initial conditions
strt = top.max() * np.ones((nlay, nrow, ncol))
ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)

# node property flow package
npf = flopy.mf6.ModflowGwfnpf(
    gwf, 
    k=hydraulic_conductivity, 
)

# storage package
sto = flopy.mf6.ModflowGwfsto(
    gwf, 
    ss=specific_storage,
)

# constant head package
# place constant heads around grid perimeter
ibd = np.ones((nlay, nrow, ncol), dtype=int)
ibd[:, 1:-1, 1:-1] = 0
idx = np.where(ibd == 1)
spd = [(k, i, j, strt[k, i, j]) for k, i, j in zip(*idx)]
chd = flopy.mf6.ModflowGwfchd(
    gwf, 
    stress_period_data=spd,
)

# wel package
w = get_well_data("HB-1")
pump_ij = model_grid.intersect(w["easting(m)"], w["northing(m)"])
spd = {0: [[(0, pump_ij[0], pump_ij[1]), -pumping_rate]], 1:[]}
wel = flopy.mf6.ModflowGwfwel(
    gwf, 
    stress_period_data=spd,
)

# output control package
budget_file = name + '.bud'
head_file = name + '.hds'
oc = flopy.mf6.ModflowGwfoc(gwf,
                            budget_filerecord=budget_file,
                            head_filerecord=head_file,
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])

# write input files
sim.write_simulation()

# run the simulation
sim.run_simulation()

# read the heads and calculate drawdown
head_obj = gwf.output.head()
head = head_obj.get_alldata()
drawdown = gwf.ic.strt.array - head

In [ ]:
fig, ax = plt.subplots()
pmv = flopy.plot.PlotMapView(gwf, ax=ax)
cb = pmv.plot_array(
    drawdown[-1]
)
# pmv.plot_grid()
plotx = 200.
ploty = 200.
ax.set_xlim(grid_center_x - plotx, grid_center_x + plotx)
ax.set_ylim(grid_center_y - ploty, grid_center_y + plotx)
plt.colorbar(cb, shrink=0.5)

syms = ["rx", "k+", 'g^']
for isym, name in enumerate(["HB-1", "HB-2", "HB-3"]):
    w = get_well_data(name)
    x, y = w["easting(m)"], w["northing(m)"]
    ax.plot(x, y, syms[isym])
    ax.text(x, y, name)

In [ ]:
# Create an array with time in the first column,
# and simulated head at the HB wells in columns
# 1, 2, and 3
idx = []
for name in ["HB-1", "HB-2", "HB-3"]:
    w = get_well_data(name)
    i, j = model_grid.intersect(w["easting(m)"], w["northing(m)"])
    idx.append((0, i, j))
time_series = head_obj.get_ts(idx)

# make a drawdown-time plot for the three HB wells
fig, ax = plt.subplots()
markers = ["o", "^", "x"]
for its, name in enumerate(["HB-1", "HB-2", "HB-3"]):
    w = get_well_data(name)
    i, j = model_grid.intersect(w["easting(m)"], w["northing(m)"])
    t = top[i, j]
    ax.plot(
        time_series[:, 0] * 1440, 
        (t - time_series[:, its + 1]), 
        ls="", 
        marker=markers[its], 
        markerfacecolor="none",
        label=name,
    )

ax.set_xscale("log")
ax.set_xlabel("time, in minutes")
ax.set_ylabel(f"drawdown, in meters")
ax.legend()

In [ ]:
# create a simple theis function
def theis_drawdown(Q, T, r, S, t):
    u = r ** 2 * S / 4. / T / t
    wu = scipy.special.exp1(u)
    s = Q / 4. / np.pi / T * wu
    return s

# calculate distance bewteen HB-1 and HB-2
w = get_well_data("HB-2")
hb2x, hb2y = w["easting(m)"], w["northing(m)"]
distance_hb1_hb2 = ((hb2x - grid_center_x) ** 2 + (hb2y - grid_center_y) ** 2) ** 0.5
print(f"{distance_hb1_hb2=}")

# test the function with some different values
Q = pumping_rate # cubic meters per day
T = transmissivity # meters squared per day
r = distance_hb1_hb2 # meters
S = storage_coefficient # unitless
t = pumping_time # days
theis_drawdown(Q, T, r, S, t)


In [ ]:
# Closer look at HB-2
fig, ax = plt.subplots()

# plot measured drawdown for HB-2
ax.plot(
    hb2_df["pumping_time(min)"], 
    hb2_df[f"drawdown(m)"],
    label="HB-2 measured",
    marker="^",
    markerfacecolor="none",
    markeredgecolor="orange",
    linestyle="",
)

# plot MODFLOW results for HB-2
w = get_well_data("HB-2")
i, j = model_grid.intersect(w["easting(m)"], w["northing(m)"])
t = top[i, j]
ax.plot(
    time_series[:, 0] * 1440, 
    t - time_series[:, 2], 
    ls="", 
    marker="o", 
    markerfacecolor="none",
    markeredgecolor="blue",
    label="HB-2 MODFLOW",
)

# plot Theis drawdown for HB-2
t = hb2_df["pumping_time(min)"]
theis = theis_drawdown(Q, T, r, S, t / 1440.)
ax.plot(
    t, 
    theis,
    label="HB-2 Theis",
    marker="none",
    color="k",
)

ax.set_xscale("log")
ax.set_xlabel("time, in minutes")
ax.set_ylabel(f"drawdown, in meters")
ax.legend()

## Exercise Questions

1.  Adjust the model hydraulic conductivity and specific storage until you get a fit between the modeled and measured drawdown.  What values give you the best match?

2.  The MODFLOW model closely matches the Theis analytical solution.  Is this expected?  Under what conditions would the MODFLOW model and Theis differ?

3.  Based on your first-hand experience with the aquifer test, what might you adjust in the model to try and better represent the measured drawdown response?